# Create Views

To facilitate ease of searching for songs to recommend users, we create views.

Each view contains the most frequently listened-to genres for each age-group and mental-health condition pair, ranked in order of frequency.

To recommend tracks to users, the (recommend_tracks) script will pick a track from the appropriate view.

### Import libraries

In [1]:
import psycopg2
import pandas as pd
import time
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access environment variables
db_elephant=os.environ.get('db_elephant')
db_elephant_dbname=os.environ.get('db_elephant_dbname')
db_elephant_dbuser=os.environ.get('db_elephant_dbuser')
db_elephant_password=os.environ.get('db_elephant_password')

### Connect to DB

In [8]:
# Connect to PostgreSQL database
db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": db_elephant_dbname, 
    "user": db_elephant_dbuser, 
    "password": db_elephant_password
}

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

### Create Views - Ranked genre Per age_group Per condition 

In [9]:
start = time.process_time()

# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Declare age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Create Views for each condition and age_group, ranking genres
for i in range(len(conditions)):
    for n in range(len(groups)):
        query = f"""
            CREATE OR REPLACE VIEW {groups[n]}_{conditions[i]}_genre AS(
                SELECT a.genre, COUNT(*),
                  ROW_NUMBER() OVER(ORDER BY COUNT(*) DESC) AS row_n
                FROM (
                  SELECT s.id, g.*
                  FROM music_survey s
                  CROSS JOIN LATERAL (
                     values (s.freq1, 'freq1'),
                            (s.freq2, 'freq2'),
                            (s.freq3, 'freq3')
                        ) as g(genre, frequency)
                WHERE s.{conditions[i]} >= 7
                AND age_group = '{groups[n]}') AS a
                WHERE a.genre IS NOT NULL
                GROUP BY a.genre
                )
                """
        cur.execute(query,)
        conn.commit()

print(time.process_time() - start)

0.0


### Check Views Created

In [10]:
# Check that views were created
query = f"""
            SELECT table_name
            FROM information_schema.views
            WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
            AND table_name !='pg_stat_statements'
            ORDER BY table_name
            """
cur.execute(query,)
conn.commit()
results = cur.fetchall()
print(*results, sep='\n')

('early_years_anxiety_genre',)
('early_years_depression_genre',)
('early_years_insomnia_genre',)
('early_years_ocd_genre',)
('elderly_anxiety_genre',)
('elderly_depression_genre',)
('elderly_insomnia_genre',)
('elderly_ocd_genre',)
('mature_adults_anxiety_genre',)
('mature_adults_depression_genre',)
('mature_adults_insomnia_genre',)
('mature_adults_ocd_genre',)
('middle_age_anxiety_genre',)
('middle_age_depression_genre',)
('middle_age_insomnia_genre',)
('middle_age_ocd_genre',)
('young_adults_anxiety_genre',)
('young_adults_depression_genre',)
('young_adults_insomnia_genre',)
('young_adults_ocd_genre',)


# Close connection

In [11]:
# close connection to db
conn.close()